In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Query 4") \
    .getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
234,application_1738075734771_0235,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Case 1: 2 Executors × 1 Core/2GB Memory

In [2]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.cores": "1",
        "spark.executor.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
235,application_1738075734771_0236,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
145,application_1738075734771_0146,pyspark,idle,Link,Link,None,
148,application_1738075734771_0149,pyspark,idle,Link,Link,None,
155,application_1738075734771_0156,pyspark,idle,Link,Link,None,
156,application_1738075734771_0157,pyspark,idle,Link,Link,None,
161,application_1738075734771_0162,pyspark,idle,Link,Link,None,
165,application_1738075734771_0166,pyspark,idle,Link,Link,None,
170,application_1738075734771_0171,pyspark,idle,Link,Link,None,
171,application_1738075734771_0172,pyspark,idle,Link,Link,None,
173,application_1738075734771_0174,pyspark,idle,Link,Link,None,
174,application_1738075734771_0175,pyspark,idle,Link,Link,None,


#### Case 2: 2 Executors × 2 Cores/4GB Memory

In [13]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.cores": "2",
        "spark.executor.memory": "4g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
236,application_1738075734771_0237,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
148,application_1738075734771_0149,pyspark,idle,Link,Link,None,
155,application_1738075734771_0156,pyspark,idle,Link,Link,None,
156,application_1738075734771_0157,pyspark,idle,Link,Link,None,
161,application_1738075734771_0162,pyspark,idle,Link,Link,None,
165,application_1738075734771_0166,pyspark,idle,Link,Link,None,
170,application_1738075734771_0171,pyspark,idle,Link,Link,None,
171,application_1738075734771_0172,pyspark,idle,Link,Link,None,
173,application_1738075734771_0174,pyspark,idle,Link,Link,None,
174,application_1738075734771_0175,pyspark,idle,Link,Link,None,
175,application_1738075734771_0176,pyspark,idle,Link,Link,None,


#### Case 3: 2 Executors × 4 Cores/8GB Memory

In [25]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.cores": "4",
        "spark.executor.memory": "8g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
237,application_1738075734771_0238,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
148,application_1738075734771_0149,pyspark,idle,Link,Link,None,
155,application_1738075734771_0156,pyspark,idle,Link,Link,None,
156,application_1738075734771_0157,pyspark,idle,Link,Link,None,
161,application_1738075734771_0162,pyspark,idle,Link,Link,None,
165,application_1738075734771_0166,pyspark,idle,Link,Link,None,
170,application_1738075734771_0171,pyspark,idle,Link,Link,None,
171,application_1738075734771_0172,pyspark,idle,Link,Link,None,
173,application_1738075734771_0174,pyspark,idle,Link,Link,None,
174,application_1738075734771_0175,pyspark,idle,Link,Link,None,
175,application_1738075734771_0176,pyspark,idle,Link,Link,None,


In [26]:

master = spark.sparkContext.master
print(f"Spark Master: {master}")

conf = spark.sparkContext.getConf()
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Cores:", conf.get("spark.executor.cores"))
print("Executor Memory:", conf.get("spark.executor.memory"))

crime_data_path_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark Master: yarn
Executor Instances: 2
Executor Cores: 4
Executor Memory: 8g

In [27]:
from sedona.spark import *
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

# Create spark Session
spark = SparkSession.builder \
    .appName("GeoJSON read") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)
# Read the file from s3
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")
# Formatting magic
flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")
# Print schema
flattened_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- BG10: string (nullable = true)
 |-- BG10FIP10: string (nullable = true)
 |-- BG12: string (nullable = true)
 |-- CB10: string (nullable = true)
 |-- CEN_FIP13: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- CITYCOM: string (nullable = true)
 |-- COMM: string (nullable = true)
 |-- CT10: string (nullable = true)
 |-- CT12: string (nullable = true)
 |-- CTCB10: string (nullable = true)
 |-- HD_2012: long (nullable = true)
 |-- HD_NAME: string (nullable = true)
 |-- HOUSING10: long (nullable = true)
 |-- LA_FIP10: string (nullable = true)
 |-- OBJECTID: long (nullable = true)
 |-- POP_2010: long (nullable = true)
 |-- PUMA10: string (nullable = true)
 |-- SPA_2012: long (nullable = true)
 |-- SPA_NAME: string (nullable = true)
 |-- SUP_DIST: string (nullable = true)
 |-- SUP_LABEL: string (nullable = true)
 |-- ShapeSTArea: double (nullable = true)
 |-- ShapeSTLength: double (nullable = true)
 |-- ZCTA10: string (nullable = true)
 |-- geometry: geometry (nulla

In [28]:
from pyspark.sql.functions import regexp_replace, col, substring, year, to_date, expr, sum, avg

# Filter Los Angeles areas based on CITY column
LA_areas = (flattened_df
    .filter(col("CITY") == "Los Angeles")
    .groupBy("COMM", "ZCTA10", "POP_2010", "HOUSING10")
    .agg(ST_Union_Aggr("geometry").alias("geometry"))
)

# Load income data from the CSV file
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv'
income_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Clean and convert the "Estimated Median Income" column from string format "$33,887" to double format 33887.0
income_df = (income_df
    .withColumn("Estimated Median Income", 
        regexp_replace(col("Estimated Median Income"), "[$,]", "").cast("double")
    )
)

# Convert "Zip Code" column to double and rename it to "ZipCode"
income_df = income_df.withColumn("ZipCode", col("Zip Code").cast("double"))

# Convert the "ZCTA10" column in Los Angeles areas to double
LA_areas = LA_areas.withColumn("ZCTA10", col("ZCTA10").cast("double"))



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
joined_df = (LA_areas.join(
    income_df,
    LA_areas.ZCTA10 == income_df.ZipCode, "inner"
).select("COMM", "ZCTA10", "POP_2010", "HOUSING10", *income_df.columns))

# Calculate total population and total number of households per ZIP Code
zip_totals = (joined_df.groupBy("ZCTA10").agg(
    sum("POP_2010").alias("TOTAL_ZIP_POP"),
    sum("HOUSING10").alias("TOTAL_ZIP_HOUSING")
))

# Join the total data (population and housing) with the main DataFrame
joined_with_totals = joined_df.join(zip_totals, "ZCTA10", "left")


# Group by area (COMM) and calculate estimated income per person
result_df = (joined_with_totals.groupBy("COMM").agg(
    sum("TOTAL_ZIP_HOUSING").alias("TOTAL_HOUSING"),
    sum("TOTAL_ZIP_POP").alias("TOTAL_POP"),
    avg("Estimated Median Income").alias("AVG_MEDIAN_INCOME")
).withColumn("Estimated_Income_Per_Person",
    (col("AVG_MEDIAN_INCOME") * col("TOTAL_HOUSING")) / col("TOTAL_POP")
))

# Filter to remove areas with NULL values
result_df = result_df.filter(result_df.Estimated_Income_Per_Person.isNotNull())

# Display the results
result_df.select("COMM", "Estimated_Income_Per_Person").show(result_df.count(), truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+---------------------------+
|COMM                   |Estimated_Income_Per_Person|
+-----------------------+---------------------------+
|Toluca Terrace         |22701.099766722724         |
|Elysian Park           |13520.428730086274         |
|Longwood               |13852.235490552479         |
|Green Meadows          |7842.072839405916          |
|Cadillac-Corning       |29557.70140860528          |
|Mid-city               |18130.855262337987         |
|Lincoln Heights        |11436.728718313136         |
|Van Nuys               |15172.370537953393         |
|Gramercy Place         |13947.49027802478          |
|Faircrest Heights      |23758.816259687963         |
|Boyle Heights          |8874.9240057775            |
|Lafayette Square       |17957.093065372475         |
|Granada Hills          |26476.89020369152          |
|North Hills            |16556.421870752303         |
|Northridge             |21928.626085734853         |
|Wilshire Center        |156

In [30]:
from pyspark.sql.functions import col, desc, asc

# Get the 3 areas with the most average income per person
top_areas_df = result_df.orderBy(desc("Estimated_Income_Per_Person")).limit(3)

# Get the 3 areas with the least average income per person
bottom_areas_df = result_df.orderBy(asc("Estimated_Income_Per_Person")).limit(3)

# Display the top 3 areas
print("Top 3 areas with the most average income per person:")
top_areas_df.select("COMM", "Estimated_Income_Per_Person").show(truncate=False)

# Display the bottom 3 areas
print("Bottom 3 areas with the least average income per person:")
bottom_areas_df.select("COMM", "Estimated_Income_Per_Person").show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Top 3 areas with the most average income per person:
+-----------------+---------------------------+
|COMM             |Estimated_Income_Per_Person|
+-----------------+---------------------------+
|Bel Air          |70929.8761562331           |
|Pacific Palisades|70039.5048525985           |
|Beverly Crest    |69775.50791751084          |
+-----------------+---------------------------+

Bottom 3 areas with the least average income per person:
+---------------+---------------------------+
|COMM           |Estimated_Income_Per_Person|
+---------------+---------------------------+
|Vernon Central |6359.233818453075          |
|Central        |6801.354588126208          |
|University Park|6872.3452079057315         |
+---------------+---------------------------+

In [31]:
file_path = "s3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv"

# Read the CSV file into a DataFrame
RE_data_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
RE_data_df.show(truncate=False)

# Print schema to inspect data types
RE_data_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------------------------+
|Vict Descent|Vict Descent Full             |
+------------+------------------------------+
|A           |Other Asian                   |
|B           |Black                         |
|C           |Chinese                       |
|D           |Cambodian                     |
|F           |Filipino                      |
|G           |Guamanian                     |
|H           |Hispanic/Latin/Mexican        |
|I           |American Indian/Alaskan Native|
|J           |Japanese                      |
|K           |Korean                        |
|L           |Laotian                       |
|O           |Other                         |
|P           |Pacific Islander              |
|S           |Samoan                        |
|U           |Hawaiian                      |
|V           |Vietnamese                    |
|W           |White                         |
|X           |Unknown                       |
|Z           |AsianIndian         

In [32]:
from pyspark.sql.functions import col, to_date, year, expr, sum
import time

# Load crime data from the specified CSV file
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv'
crimeData_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Extract the year from the "Date Rptd" column and filter for crimes reported in 2015
crimeData_df = crimeData_df.withColumn("year", year(to_date(col("Date Rptd"), "MM/dd/yyyy hh:mm:ss a")))
crimeData_df = crimeData_df.filter(col("year") == 2015)

# Create a geometry column using the latitude (LAT) and longitude (LON) values
crimeData_df = crimeData_df.withColumn("geom", expr("ST_Point(LON, LAT)"))

# Start measuring execution time
start_time = time.time()

# Perform a spatial join between crime data and Los Angeles area geometries
crimeDataGEO_df = crimeData_df.hint("SHUFFLE_REPLICATE_NL").join(
    LA_areas.hint("SHUFFLE_REPLICATE_NL"), expr("ST_Within(geom, geometry)"), "inner"
)

# Group by community (COMM) and calculate the total number of crimes per community
community_crime_stats = crimeDataGEO_df.groupBy("COMM").agg(
    expr("count(*)").alias("Total_Crimes_Per_Community")
)

# Aggregate the total population for each community
community_population = LA_areas.groupBy("COMM").agg(
    sum("POP_2010").alias("TOTAL_COMM_POP")
)

# Join the crime statistics with the population data on community (COMM)
final_stats = community_crime_stats.hint("SHUFFLE_REPLICATE_NL").join(
    community_population.hint("SHUFFLE_REPLICATE_NL"), "COMM", "inner"
)

# Compute the crime rate per person for each community
final_stats = final_stats.withColumn(
    "Crimes_Per_Person", col("Total_Crimes_Per_Community") / col("TOTAL_COMM_POP")
)

# Calculate and display the execution time
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")

# Show the final computed statistics
final_stats.show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time: 0.0895 seconds
+-------------------+--------------------------+--------------+--------------------+
|COMM               |Total_Crimes_Per_Community|TOTAL_COMM_POP|Crimes_Per_Person   |
+-------------------+--------------------------+--------------+--------------------+
|Venice             |2949                      |30663         |0.09617454260835534 |
|Playa Vista        |445                       |8926          |0.049854358055119875|
|Westchester        |3741                      |46196         |0.08098103731924843 |
|Del Rey            |1111                      |27390         |0.040562248995983936|
|Hyde Park          |2190                      |26576         |0.0824051776038531  |
|Marina Peninsula   |185                       |4120          |0.044902912621359224|
|View Heights       |145                       |3257          |0.04451949646914338 |
|Cheviot Hills      |371                       |8761          |0.042346764068028765|
|West Los Angeles   |1769         

In [33]:
crime_data_1 = spark.read.csv(
    path=file_path,
    header=True,
    inferSchema=True
).select("LON", "LAT", "DATE OCC", "Vict Descent")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
blocks_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geometry: geometry (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- BG10: string (nullable = true)
 |    |-- BG10FIP10: string (nullable = true)
 |    |-- BG12: string (nullable = true)
 |    |-- CB10: string (nullable = true)
 |    |-- CEN_FIP13: string (nullable = true)
 |    |-- CITY: string (nullable = true)
 |    |-- CITYCOM: string (nullable = true)
 |    |-- COMM: string (nullable = true)
 |    |-- CT10: string (nullable = true)
 |    |-- CT12: string (nullable = true)
 |    |-- CTCB10: string (nullable = true)
 |    |-- HD_2012: long (nullable = true)
 |    |-- HD_NAME: string (nullable = true)
 |    |-- HOUSING10: long (nullable = true)
 |    |-- LA_FIP10: string (nullable = true)
 |    |-- OBJECTID: long (nullable = true)
 |    |-- POP_2010: long (nullable = true)
 |    |-- PUMA10: string (nullable = true)
 |    |-- SPA_2012: long (nullable = true)
 |    |-- SPA_NAME: string (nullable = true)
 |    |-- SUP_DIST: string (nullable = true)
 |    |-

In [35]:
from pyspark.sql.functions import to_timestamp, year, when, col
import time

start_time = time.time()

# Step 2: Convert DATE OCC to date format
crime_data = crime_data_1.withColumn("DATE_OCC", to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a"))

# Step 3: Filter for 2015 crimes
crimes_2015 = crime_data.filter(year(col("DATE_OCC")) == 2015)

# Step 4: Add geometry column
crimes_2015 = crimes_2015.withColumn("geometry", ST_Point(col("LON"), col("LAT")))

# Step 5: Spatial join for specific areas
specific_areas_1 = ["Bel Air", "Beverly Crest", "Pacific Palisades"]
specific_areas_2 = ["Vernon", "Central", "University Park"]

crimes_2015.createOrReplaceTempView("crime_data_2015")
blocks_df.createOrReplaceTempView("area_boundaries")

spatial_join_areas_1 = spark.sql(f"""
    SELECT b.properties.COMM AS area_name, c.`Vict Descent` AS descent
    FROM crime_data_2015 c
    JOIN area_boundaries b
    ON ST_Within(c.geometry, b.geometry)
    WHERE b.properties.COMM IN ({', '.join([f"'{area}'" for area in specific_areas_1])})
""")

spatial_join_areas_2 = spark.sql(f"""
    SELECT b.properties.COMM AS area_name, c.`Vict Descent` AS descent
    FROM crime_data_2015 c
    JOIN area_boundaries b
    ON ST_Within(c.geometry, b.geometry)
    WHERE b.properties.COMM IN ({', '.join([f"'{area}'" for area in specific_areas_2])})
""")


# Step 6: Group by victim ethnic background for both sets of areas
crime_by_descent_areas_1 = spatial_join_areas_1.groupBy("descent").count()
crime_by_descent_areas_2 = spatial_join_areas_2.groupBy("descent").count()

# Step 7: Map descent codes to descriptions
descent_codebook = {
    "A": "Other Asian",
    "B": "Black",
    "C": "Chinese",
    "D": "Cambodian",
    "F": "Filipino",
    "G": "Guamanian",
    "H": "Hispanic/Latin/Mexican",
    "I": "American Indian/Alaskan Native",
    "J": "Japanese",
    "K": "Korean",
    "L": "Laotian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Samoan",
    "U": "Hawaiian",
    "V": "Vietnamese",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian Indian",
}

crime_by_descent_areas_1 = crime_by_descent_areas_1.withColumn(
    "descent_description",
    when(
        col("descent").isin(list(descent_codebook.keys())),
        col("descent")
    ).otherwise("Unknown")
).replace(descent_codebook, subset=["descent_description"])

crime_by_descent_areas_2 = crime_by_descent_areas_2.withColumn(
    "descent_description",
    when(
        col("descent").isin(list(descent_codebook.keys())),
        col("descent")
    ).otherwise("Unknown")
).replace(descent_codebook, subset=["descent_description"])

# Step 8: Display results
# Order the results by the count column in descending order
crime_by_descent_areas_1_ordered = crime_by_descent_areas_1.orderBy(col("count").desc())
crime_by_descent_areas_2_ordered = crime_by_descent_areas_2.orderBy(col("count").desc())

# Display the ordered results for both sets of areas
print("Results for Bel Air, Beverly Crest, Pacific Palisades:")
crime_by_descent_areas_1_ordered.select("descent_description", "count").show(truncate=False)

print("Results for Vernon, Central, University Park:")
crime_by_descent_areas_2_ordered.select("descent_description", "count").show(truncate=False)

end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Results for Bel Air, Beverly Crest, Pacific Palisades:
+------------------------------+-----+
|descent_description           |count|
+------------------------------+-----+
|White                         |918  |
|Other                         |199  |
|Hispanic/Latin/Mexican        |95   |
|Unknown                       |56   |
|Black                         |53   |
|Unknown                       |38   |
|Other Asian                   |33   |
|Chinese                       |1    |
|American Indian/Alaskan Native|1    |
+------------------------------+-----+

Results for Vernon, Central, University Park:
+------------------------------+-----+
|descent_description           |count|
+------------------------------+-----+
|Hispanic/Latin/Mexican        |1775 |
|Black                         |416  |
|White                         |403  |
|Unknown                       |344  |
|Other                         |218  |
|Other Asian                   |138  |
|American Indian/Alaskan Native|24   |
|